In [ ]:
# allows update of external libraries without need to reload package
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import pickle
import matplotlib.pyplot as plt
import pathlib
import numpy as np
import warnings
import xarray

import sklearn.metrics
import sklearn.model_selection
import scipy.stats

In [ ]:
FOLDER_DATA = pathlib.Path("../Data/Task1/Original/")
FILE_TWEETS = "/home/kristian/Projects/a2/data/tweets/2020_tweets_rain_sun_vocab_emojis_locations_bba_Tp_era5_no_bots_normalized_filtered.nc"
FOLDER_TWEETS_MODEL_DATA = pathlib.Path("../Data/Task4/Original/")

In [ ]:
def load_pickle(filename):
    with open(filename, "rb") as input_file:
        return pickle.load(input_file)
    


In [ ]:
for filename in os.listdir(FOLDER_DATA):
    data = load_pickle(filename=FOLDER_DATA / filename)
    print(f"{filename=}\n{data[0]}\n{data[-1]}")
type(data)

In [ ]:
tweets = xarray.load_dataset(FILE_TWEETS)

In [ ]:
tweets.text_normalized.values

In [ ]:
np.all([x==y for x, y in zip(tweets.tp_h.values>1e-6, tweets.raining.values)])

In [ ]:
def save_pickle(filename, to_save):
    os.makedirs(filename.parent, exist_ok=True)
    with open(filename, 'wb') as handle:
        pickle.dump(to_save, handle)


In [ ]:
X = tweets.text_normalized.values
y = tweets.raining.values

In [ ]:
indices_train, indices_test_dev = sklearn.model_selection.train_test_split(
    np.arange(tweets["index"].shape[0]),
    test_size=0.4,
    random_state=42,
    shuffle=True,
    stratify=tweets.raining.values,
)

indices_dev, indices_test = sklearn.model_selection.train_test_split(
    indices_test_dev,
    test_size=0.5,
    random_state=42,
    shuffle=True,
    stratify=tweets.raining.values[indices_test_dev],
)


In [ ]:
def convert_features_labels_to_list(features, labels):
    return [[f, l] for f, l in zip(features, labels)]
    
    
for indices, filename in zip([indices_train, indices_dev, indices_test], ['train.pkl', 'dev.pkl', 'test_in.pkl']):
    save_pickle(FOLDER_TWEETS_MODEL_DATA / filename, convert_features_labels_to_list(X[indices], y[indices]))